In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize
import pickle
import numpy as np
from keras.api.preprocessing.sequence import pad_sequences
test = pd.read_csv("work-data/test_nolabel.csv")

In [2]:
def tokenize_text(text):
    return [
        [word.lower() for word in word_tokenize(sentence)] for sentence in sent_tokenize(text)
    ]


In [3]:
tokenized_test = [sent for item in test['text'].apply(tokenize_text) for sent in item]
word_index = pickle.load(open("models/word_index.pkl", "rb"))

In [4]:
def text_to_seq(text, word_index):
    return [word_index[word] for word in word_tokenize(text.lower()) if word in word_index]

max_len = pickle.load(open("models/max_len.pkl", "rb"))
X_test_seq = [text_to_seq(text, word_index) for text in test['text']]
X_test_padded = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

print(X_test_padded)


[[  45   22  134 ...    0    0    0]
 [  26 1126  709 ...    0    0    0]
 [4121    9    4 ...    0    0    0]
 ...
 [ 384 1558  201 ...    0    0    0]
 [   3    3    5 ...    0    0    0]
 [  61    6   10 ...    0    0    0]]


In [5]:
from keras.api.models import load_model

model = load_model("models/lstm-model.keras", compile=False)
y_predict = model.predict(X_test_padded)

2025-03-07 22:43:37.353968: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4 Pro
2025-03-07 22:43:37.353988: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 24.00 GB
2025-03-07 22:43:37.353994: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 8.00 GB
I0000 00:00:1741362217.354006  150800 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1741362217.354021  150800 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-03-07 22:43:38.014899: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step


In [6]:
label = np.argmax(y_predict, axis=-1)

label_encoder = pickle.load(open("models/label_encoder.pkl", "rb"))

test['label'] = label_encoder.inverse_transform(label)



In [7]:
test[['id', 'label']].to_csv("work-data/test_label.csv", index=False)